### call utility functions to get the analysis file

In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os

import sys
script_directory = '/Users/joshwinnes/Library/Mobile Documents/com~apple~CloudDocs/Wheaton College/fall 2024/topics in data science/Math485/packages'
sys.path.append(script_directory)
import dhs_util
from dhs_util import *

os.chdir('/Users/joshwinnes/Library/Mobile Documents/com~apple~CloudDocs/Wheaton College/fall 2024/topics in data science/data')
df = pd.read_csv('dhs_service_records_synthesized_final.csv')

df = dhs_preprocessing(df)
df, service_map = add_service_label(df)
df = add_age_bin(df)

recipient = get_recipient_attribute(df)

/Users/joshwinnes/Library/Mobile Documents/com~apple~CloudDocs/Wheaton College/fall 2024/topics in data science/Math485/packages/dhs_util.py:42: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  recipient = df.groupby(df['id'].cat.remove_unused_categories()).agg(


In [7]:
print(df.shape)
df.head()

(7116134, 12)


,id,date,service,age,gender,race,ethnicity,marital,education,month,serv,age_bin
0,1626,2021-01-31,Children_Attending_Early_Childhood_Programs_Ma...,5,1~Male,1~White,99~Unknown,99~Unknown,9-12~High School (grade 9-12),January,S01,0-9
1,1626,2021-02-28,Children_Attending_Early_Childhood_Programs_Ma...,5,1~Male,1~White,99~Unknown,99~Unknown,9-12~High School (grade 9-12),February,S01,0-9
2,1626,2021-03-31,Children_Attending_Early_Childhood_Programs_Ma...,5,1~Male,1~White,99~Unknown,99~Unknown,9-12~High School (grade 9-12),March,S01,0-9
3,1626,2021-04-30,Children_Attending_Early_Childhood_Programs_Ma...,5,1~Male,1~White,99~Unknown,99~Unknown,9-12~High School (grade 9-12),April,S01,0-9
4,1626,2021-05-31,Children_Attending_Early_Childhood_Programs_Ma...,5,1~Male,1~White,99~Unknown,99~Unknown,9-12~High School (grade 9-12),May,S01,0-9


In [8]:
print(recipient.shape)
recipient.head()

(533799, 15)


,id,num_service,distinct_service,first_date,last_date,num_month,distinct_month,age,gender,race,ethnicity,marital,education,serv,age_bin
0,1,12,1,2021-01-31,2021-12-31,12,12,13,1~Male,99~Unknown,2~Not Hispanic/Latinx,99~Unknown,GED~High School Diploma/GED Completed,S12,10-19
12,2,1,1,2021-02-28,2021-02-28,1,1,70,2~Female,1~White,2~Not Hispanic/Latinx,2~Married,99~Unknown,S12,70-79
13,3,11,1,2021-02-28,2021-12-31,11,11,29,2~Female,1~White,2~Not Hispanic/Latinx,2~Married,9-12~High School (grade 9-12),S12,20-29
24,4,4,1,2021-03-31,2021-12-31,4,4,4,2~Female,1~White,2~Not Hispanic/Latinx,1~Single-Never Married,9-12~High School (grade 9-12),S12,0-9
28,5,12,1,2021-01-31,2021-12-31,12,12,40,1~Male,2~Black/African American,2~Not Hispanic/Latinx,2~Married,UNDERGRAD~Some College,S12,40-49


In [ ]:
df.date.unique()

## In class exercises
Answer the following questions:
 - 
 - do the number of services per recipient increase over time?  - What else can you observe from the data?

In [ ]:
serv_data = []
for groups in df.groupby('id').groups.values():
    serv_data.append(df.loc[groups]['serv'].tolist())
serv_data[:10]

In [ ]:
serv_data[:10]

* another way to do the same

In [ ]:
serv_data_n = []
for groups in df.groupby('id').groups.values():
    serv_data_n.append(list(df.loc[groups]['serv'].to_numpy()))
serv_data_n[:10]


* check equality

In [ ]:
serv_data == serv_data_n

### use mlxtend package

In [ ]:
from mlxtend.preprocessing import TransactionEncoder
from mlxtend.preprocessing import *
from mlxtend.frequent_patterns import association_rules
from mlxtend.frequent_patterns import fpgrowth
from mlxtend.frequent_patterns import apriori
from mlxtend.frequent_patterns import fpmax

def oneHotCoding(serv_data):
    te = TransactionEncoder()
    te_ary = te.fit(serv_data).transform(serv_data)
    te_df = pd.DataFrame(te_ary, columns=te.columns_)
    return te_df

serv_oneHot = oneHotCoding(serv_data)

In [ ]:
serv_oneHot

* we have done something like this earlier, recall? 

In [ ]:
apriori(serv_oneHot, min_support=0.018) 

In [ ]:
min_freq = 1000
min_support = min_freq/serv_oneHot.shape[0]
min_confidence = 0.6
min_rule_support = 0.2
min_lift = 0.15

In [ ]:
apriori(serv_oneHot, min_support=min_support)

In [ ]:
freq_itemsets_apriori = apriori(serv_oneHot,min_support=min_support,use_colnames=True)
freq_itemsets_apriori.describe()

In [ ]:
freq_itemsets_fpgrowth = fpgrowth(serv_oneHot,min_support=min_support,use_colnames=True)
freq_itemsets_fpgrowth.describe()

In [ ]:
freq_itemsets_fpmax = fpmax(serv_oneHot,min_support=min_support,use_colnames=True)
freq_itemsets_fpmax.describe()

In [ ]:
# compute and print the association rules
def serv_rules(freq_itemsets,metrics,threshold):
    asso_rules = association_rules(freq_itemsets, metric=metrics, min_threshold=threshold)
    return asso_rules.sort_values(by='lift', ascending=False)[['antecedents', 'consequents', 'support', 'confidence', 'lift']]

rules_apriori = serv_rules(freq_itemsets_apriori,"confidence",0.60)
rules_fpgrowth = serv_rules(freq_itemsets_fpgrowth,"confidence",0.60)

In [ ]:
rules_fpgrowth

In [ ]:
def predict(antecedent, rules, consequents_only = False):
    
    # get the rules for this antecedent
    preds = rules[rules['antecedents'] == antecedent]
    if consequents_only:
        # a way to convert a frozen set with one element to string
        preds = preds['consequents'].apply(iter).apply(next)
    return preds

In [ ]:
rules_fpmax = association_rules(freq_itemsets_fpmax, metric="confidence", min_threshold=0.001, support_only=True)

In [ ]:
predict({"S09"}, rules_fpmax, consequents_only=False)


In [ ]:
predict({"S09"}, rules_fpmax, consequents_only=False)


In [ ]:
predict({"S09"}, rules_fpgrowth)


In [ ]:
len(predict({"S01"},rules_fpgrowth))

In [ ]:
serv_list =  ['S'+str(i).zfill(2) for i in range(1,23)]
for i in serv_list:
    print(i)
    if (len(predict({i},rules_fpgrowth))>0):
        print(i), print(predict({i},rules_fpgrowth))

In [ ]:
rules_fpgrowth.info()

In [ ]:
rules_fpgrowth.sort_values("support",ascending=False)


In [ ]:
df[(df.serv=="S09")].merge(df[(df.serv =="S12")], on = "id").id.nunique()/df.id.nunique()

In [ ]:
#make a scatterplot between age and number of services in the client_attributes dataset

sns.scatterplot(data = df, x = 'age', y = 'race', s = 2)
plt.title('Age vs Number of Services')
plt.xlabel('Age')
plt.ylabel('Number of Services')
plt.show()

In [ ]:
import itertools
j = 0
for i in itertools.combinations(df.service.unique(),2):
    j=j+1
    print(j), print(i)